<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Useful-Inputs" data-toc-modified-id="Useful-Inputs-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Useful Inputs</a></span></li><li><span><a href="#Obtaining-Dataset-&amp;-Train-Test-Split" data-toc-modified-id="Obtaining-Dataset-&amp;-Train-Test-Split-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Obtaining Dataset &amp; Train-Test Split</a></span></li><li><span><a href="#Creating-a-Neural-Network" data-toc-modified-id="Creating-a-Neural-Network-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Creating a Neural Network</a></span></li><li><span><a href="#Training-the-Model" data-toc-modified-id="Training-the-Model-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Training the Model</a></span></li><li><span><a href="#Evaluating-the-Trained-Model" data-toc-modified-id="Evaluating-the-Trained-Model-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Evaluating the Trained Model</a></span></li><li><span><a href="#Saving-it-for-later" data-toc-modified-id="Saving-it-for-later-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Saving it for later</a></span></li></ul></div>

Most of this code can be found at <a href='https://keras.io'>keras.io</a>

# Useful Inputs

In [ ]:
from tensorflow import keras
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np

np.random.seed(27)

In [ ]:
from keras import models
from keras import layers
from keras import optimizers

# Obtaining Dataset & Train-Test Split

Nothing different from training other models

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler

iris = load_iris()
x = iris['data']
y = iris['target']
names = iris['target_names']
feature_names = iris['feature_names']

# One hot encoding
enc = OneHotEncoder()
y = enc.fit_transform(y[:, np.newaxis]).toarray()

# Scale data to have mean 0 and variance 1 
# which is importance for convergence of the neural network
scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)

# Split the data set into training, validation, and testing
x_train, x_test, y_train, y_test = train_test_split(
                                    x_scaled, y, 
                                    test_size=0.2, random_state=2
)
cut_off = int(len(x_train) * 0.9)

x_valid, x_train = x_train[:cut_off] , x_train[cut_off:] 
y_valid, y_train = y_train[:cut_off], y_train[cut_off:]

# Creating a Neural Network

`Sequential` is referring to the neural networks we've observed. There are other neural network models that will go beyond this class.

In [ ]:
from keras.models import Sequential

model = Sequential()

The actual network; we can decide how many layers & nodes for each layer here as well as other hyperparameters like the activation function.

In [ ]:
from keras.layers import Dense

model.add(Dense(units=30, activation='relu', input_dim=4))
# Use a 2nd hidden layer for more parameters & complexity
# model.add(Dense(units=16, activation='relu'))
model.add(Dense(units=3, activation='softmax'))

Compile the model to a form that the computer can more easily work with

In [ ]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=optimizers.SGD(lr=0.001, momentum=0.9, nesterov=True),
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
model.layers

In [ ]:
model.get_layer('dense_1')

In [ ]:
weights, biases = model.layers[0].get_weights()

print('Weights:\n', weights)
print()
print('Biases:\n', biases)

# Training the Model

Using the model structure, we do sequences of feedfoward and then backpropagation to adjust the weights and biases (training/fitting)

In [ ]:
history = model.fit(x_train, y_train, 
                    epochs=500, #batch_size=256,
                    validation_data = (x_valid, y_valid)
                    # validation_split=0.2 # Optionally use a split here
)

In [ ]:
# alternatively could have specified a specific batch to train on
# model.train_on_batch(x_batch, y_batch)

In [ ]:
weights, biases = model.layers[0].get_weights()

print('Weights:\n', weights)
print()
print('Biases:\n', biases)

# Evaluating the Trained Model

In [ ]:
df = pd.DataFrame(history.history)
columns = ['acc', 'val_acc']

df[columns].plot(figsize=(8, 5))
plt.grid(True)
plt.show()

In [ ]:
df = pd.DataFrame(history.history)
columns = ['loss', 'val_loss']

df[columns].plot(figsize=(8, 5))
plt.grid(True)
plt.show()

We can look at the overall loss from our test data after training the model was trained

In [ ]:
# Could also just use a batch to evaluate
# loss_and_metrics = model.evaluate(x_test, y_test, batch_size=16)

loss_and_metrics = model.evaluate(x_test, y_test)

loss_and_metrics

We can have predictions (probability the data point is a particular class based on our trained model)

In [ ]:
classes = model.predict(x_test)

classes

We want to say what is the predicted class, so we pick just the largest probability for each result

In [ ]:
predictions = np.argmax(classes, axis=1)

predictions

Finally, we can see how accurate our model was by seeing if the predicted classes match the actual labels. Note that this is calculated differently from how the loss is calculated.

In [ ]:
np.sum(predictions  == np.argmax(y_test, axis=1)) / predictions.shape

# Saving it for later

In [ ]:
model.save("my_model.h5")

In [ ]:
loaded_model = keras.models.load_model("my_model.h5")